In [ ]:
import pandas as pd
df = pd.read_csv('CAR DETAILS FROM CAR DEKHO.csv')
df

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df['seller_type'].unique())
print(df['transmission'].unique())
print(df['owner'].unique())
print(df['fuel'].unique())

In [ ]:
#Check the missing value and null value
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
final_dataset=df[['year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner']]
final_dataset.head()

In [ ]:
final_dataset['Current_Year']=2022
final_dataset.head()

In [ ]:
final_dataset['no_year']=final_dataset['Current_Year']-final_dataset['year']
final_dataset.head()

In [ ]:
final_dataset.drop(['year'],inplace=True,axis=1)
final_dataset.head()

In [ ]:
final_dataset.drop(['Current_Year'],axis=1,inplace=True)
final_dataset.head()

In [ ]:
final_dataset=pd.get_dummies(final_dataset,drop_first=True)
final_dataset.head()

In [ ]:
final_dataset=final_dataset.dropna()
final_dataset.head()

In [ ]:
final_dataset.drop(['fuel_Electric','fuel_LPG'],axis=1,inplace=True)
final_dataset.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat=final_dataset.corr()
top_corr_features=corrmat.index
plt.figure(figsize=(20,20))
#plot heat  map
g = sns.heatmap(final_dataset[top_corr_features].corr(),annot=True,cmap='RdYlGn')

In [ ]:
final_dataset.head()

In [ ]:
#independent and dependent feature
X = final_dataset.iloc[:,1:]
y = final_dataset.iloc[:,0]
X.head()

In [ ]:
y.head()

In [ ]:
#feature importance
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of fetaure importance for better visualization
feat_importances=pd.Series(model.feature_importances_,index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random=RandomForestRegressor()

In [ ]:
###Hyperparameters
import numpy as np
n_estimators = [int(x)for x in np.linspace(start=100,stop=1200,num=12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Randomized Search Cv

#Number of trees in random forest
n_estimators=[int(x)for x in np.linspace(start=100,stop=1200,num=12)]
#number of features to consider at every split
max_features=['auto','sqrt']
#maximum number of level in tree
max_depth=[int(x)for x in np.linspace(5,30,num=6)]
#max_depth.append(None)
#minimum number of samples required to split a node
min_samples_split=[2,5,10,15,100]
#minimum number of sample required at each leaf node
min_samples_leaf=[1,2,5,10]


In [ ]:
random_grid={
    'n_estimators':n_estimators,
    'max_depth':max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'max_features': max_features,
}

print(random_grid)

In [ ]:
#Use the random grid to search for best hyperparameters
#First create the base model to tune
rf=RandomForestRegressor()

In [ ]:
rf_random=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,scoring='neg_mean_squared_error',n_iter=10,cv=5,verbose=2,random_state=42,n_jobs=1)
rf_random.fit(X_train,y_train)

In [ ]:
prediction = rf_random.predict(X_test)
prediction

In [ ]:
sns.distplot(y_test-prediction)

In [ ]:
plt.scatter(y_test,prediction)

In [ ]:
import pickle
#open a file where you ant to store the data
file = open('random_forest_regression_model.pkl','wb')
#dump information to that file
pickle.dump(rf_random, file)